# Data Classes – Interactive Demo

This Jupyter Lab Notebook will be my demo scratch surface. This way I can preserv and prepare the results with nice descriptions.

First we need to import the `dataclass`-decorator

In [139]:
from dataclasses import dataclass

from datetime import datetime


Let's create a simple data class


In [140]:
@dataclass
class PyCon:
    location: str
    date: datetime
    year: int = 2018  # Default year

It's that simple, let's try it out

If we create an instance without specifying default a TypeError will be raised

In [141]:
try:
    PyCon()
except TypeError as e:
    print(e)

__init__() missing 2 required positional arguments: 'location' and 'date'


Location and date missing, the year is given the default value. We can see that the properties are translated to kwargs in the `__init__()`-function

Ok so let's try to properly instantiate it

In [142]:
# For you who are obesrvant, fromisoformat() was added to datetime in Python 3.7

pycon_se_18 = PyCon("Stockholm", datetime.fromisoformat("2018-12-12"))
pycon_se_18

PyCon(location='Stockholm', date=datetime.datetime(2018, 12, 12, 0, 0), year=2018)

That worked, but could be nicer.

Let's improve our class some
- Set the year to the current year instead of hardcoding
- Pass the date as a _iso-string_ an convert it automatically

In [143]:
from dataclasses import field
from typing import Union

@dataclass
class PyCon:
    location: str
    # Can be passed as a str or datetime
    date: Union[datetime, str]
    
    # Add a factory for this year
    year: int = field(
        default_factory=lambda: datetime.now().year
    )
    
    # post init runs after __init__ finishes, here we can add more nicities
    def __post_init__(self):
        
        if isinstance(self.date, str):
            # Will raise a ValueError if string is of invalid isoformat
            self.date = datetime.fromisoformat(self.date)

In [144]:
pycon_se_18 = PyCon("Stockholm", "2018-12-12")
pycon_se_18

PyCon(location='Stockholm', date=datetime.datetime(2018, 12, 12, 0, 0), year=2018)

Voila, much smoother then before.


# Part II
Now we got a useful class, let's see what tricks it can do.
There are a bunch of extra functions included in the dataclass module, I won't be able to cover all in this talk but let's play with some essentials.

First out, the `astuple()`-function.

In [145]:
from dataclasses import astuple
astuple(pycon_se_18)

('Stockholm', datetime.datetime(2018, 12, 12, 0, 0), 2018)

Very useful when unpacking:

In [146]:
pycon_city, _, pycon_year = astuple(pycon_se_18)

f"PyCon in {pycon_city}, {pycon_year}"

'PyCon in Stockholm, 2018'

But in my opinion dicts are even more useful.

So without further 

In [147]:
from dataclasses import asdict
asdict(pycon_se_18)

{'location': 'Stockholm',
 'date': datetime.datetime(2018, 12, 12, 0, 0),
 'year': 2018}

In [148]:
# Create a simple JSONEncoder

import json
from dataclasses import is_dataclass


class CustomEncoder(json.JSONEncoder):

   def default(self, obj):
        
        # Handle dataclasses
        if is_dataclass(obj):
            return asdict(obj)
        
        # Handle datetimes
        if isinstance(obj, datetime):
            return obj.isoformat()


Now let's try our simple decoder with our `PyCon` object, remember we did't write anything specific for `PyCon`-classes in the decoder

In [149]:
json.dumps(pycon_se_18, cls=CustomEncoder)

'{"location": "Stockholm", "date": "2018-12-12T00:00:00", "year": 2018}'

And decoding the JSON back to a PyCon-object?

In [150]:
json_payload = json.loads(json.dumps(pycon_se_18, cls=CustomEncoder))
PyCon(**json_payload)

PyCon(location='Stockholm', date=datetime.datetime(2018, 12, 12, 0, 0), year=2018)